In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [125]:
data = pd.read_csv('energydata_complete.csv')
df = data.copy()

In [126]:
pd.options.display.max_columns=None

In [127]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [128]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,19.592106,50.949283,7.910939,54.609083,20.267106,35.388200,22.029107,42.936165,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,1.844623,9.022034,6.090347,31.149806,2.109993,5.114208,1.956162,5.224361,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,15.330000,29.815000,-6.065000,1.000000,15.390000,23.200000,16.306667,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,18.277500,45.400000,3.626667,30.025000,18.700000,31.500000,20.790000,39.066667,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,19.390000,49.090000,7.300000,55.290000,20.033333,34.863333,22.100000,42.375000,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,20.619643,53.663333,11.256000,83.226667,21.600000,39.000000,23.390000,46.536000,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,25.795000,96.321667,28.290000,99.900000,26.000000,51.400000,27.230000,58.780000,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [129]:
x = df.T2

In [130]:
x_matrix = x.values.reshape(-1,1)

In [131]:
scaler = StandardScaler()
features_std = scaler.fit_transform(x_matrix)

In [132]:
y= df.T6

In [133]:
x_train,x_test,y_train,y_test = train_test_split(features_std , y, random_state = 42)

In [134]:
reg = LinearRegression().fit(x_train,y_train)

In [135]:
y_pred = reg.predict(x_test)

In [136]:
reg.score(x_train,y_train)

0.6422219138291818

In [137]:
mean_absolute_error(y_test, y_pred)

2.8204572069165827

In [138]:
reg.coef_

array([4.88447908])

In [161]:
rss = np.sum(np.square(y_test-y_pred))
round(rss,3)

65080.492

In [140]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
round(rmse,3)

3.632

In [141]:
target = df.Appliances

In [143]:
features = df.drop(['Appliances','lights','date'], axis = 1)

In [144]:
features.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [145]:
scaler = StandardScaler()
f_std = scaler.fit_transform(features)

In [146]:
xtrain,xtest,ytrain,ytest = train_test_split(f_std, target, random_state = 42)

In [147]:
regr = LinearRegression().fit(xtrain,ytrain)

In [148]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(regr.coef_, features.columns)._values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Featuers', col_name]
    weights_df[col_name].round(3)
    return weights_df


In [149]:
regr.coef_

array([  0.99223305,  68.59818508, -42.88259105, -59.6427878 ,
        51.81626111,  15.25256986,   5.252602  ,   6.3553701 ,
        -3.08842284,   1.92447259,  44.78365111,  12.63927394,
         2.93600808,  -9.6658026 ,  19.49312816, -29.72373456,
       -42.02561583,  -7.57401595, -56.06804435,   1.1739456 ,
       -13.99717983,   5.60799321,   2.42122133,  21.4587724 ,
         0.14636266,   0.14636266])

In [150]:
col_name = features.columns.values

In [151]:
weights = pd.DataFrame(data=regr.coef_, columns = ['col_name'],index = col_name)

In [152]:
weights

,col_name
T1,0.992233
RH_1,68.598185
T2,-42.882591
RH_2,-59.642788
T3,51.816261
RH_3,15.252570
T4,5.252602
RH_4,6.355370
T5,-3.088423
RH_5,1.924473


In [153]:
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(xtrain,ytrain)

C:\Users\obehi\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 324268.45751172304, tolerance: 15841.176388081885
  positive)


Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [154]:
lasso_y_pred = lasso_reg.predict(xtest)

In [155]:
lasso_reg.coef_

array([ 9.45153601e-01,  6.85673688e+01, -4.27834457e+01, -5.95649103e+01,
        5.18038112e+01,  1.52381769e+01,  5.23611296e+00,  6.33328285e+00,
       -3.09635398e+00,  1.92400662e+00,  4.46769261e+01,  1.26010198e+01,
        2.90846979e+00, -9.65163569e+00,  1.94936341e+01, -2.97304180e+01,
       -4.19995370e+01, -7.56993992e+00, -5.56763942e+01,  1.17334501e+00,
       -1.38029025e+01,  5.61455279e+00,  2.41833226e+00,  2.11917960e+01,
        2.91461392e-01,  8.32604238e-16])

In [156]:
lasso_weights = pd.DataFrame(data=lasso_reg.coef_, columns = ['col_name'],index = col_name)
lasso_weights

,col_name
T1,9.451536e-01
RH_1,6.856737e+01
T2,-4.278345e+01
RH_2,-5.956491e+01
T3,5.180381e+01
RH_3,1.523818e+01
T4,5.236113e+00
RH_4,6.333283e+00
T5,-3.096354e+00
RH_5,1.924007e+00


In [157]:
las_rmse = np.sqrt(mean_squared_error(y_test,lasso_y_pred))
round(rmse,3)

3.632

In [158]:
ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(x_train,y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [159]:
ridge_y_pred = ridge_reg.predict(x_test)

In [160]:
rmse = np.sqrt(mean_squared_error(y_test,ridge_y_pred))
round(rmse,3)

3.632